# W_GL_SEGMENT_D ETL Process
### EDW W_GL_SEGMENT_DS to W_GL_SEGMENT_D (Dimension SCD Type 2)
**ODI Package:** SILOS_SIL_GLSEGMENTDIMENSION  
**IKM:** BIAPPS Oracle Slowly Changing Dimension

In [ ]:
%sql
-- Step 1: Variable IS_INCREMENTAL
CREATE OR REPLACE TEMP VIEW VAR_IS_INCREMENTAL AS
SELECT CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SILOS_SIL_GLSEGMENTDIMENSION'
  AND (DATASOURCE_NUM_ID = 380 OR DATASOURCE_NUM_ID = 380)
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1'

In [ ]:
%sql
-- Step 2: Create Flow Table I$_2625027_1 with NOT EXISTS change detection
DROP VIEW IF EXISTS I$_2625027_1;

CREATE OR REPLACE TEMP VIEW I$_2625027_1 AS
SELECT
  0 AS SCD1_WID,
  INLINE_VIEW.SEGMENT_LOV_ID,
  COALESCE(INLINE_VIEW.SEGMENT_VAL_CODE, '__NOT_APPLICABLE__') AS SEGMENT_VAL_CODE,
  COALESCE(INLINE_VIEW.ROW_WID, 0) AS CREATED_BY_WID,
  COALESCE(LKP_USER_CHANGED.ROW_WID, 0) AS CHANGED_BY_WID,
  INLINE_VIEW.CREATED_ON_DT,
  INLINE_VIEW.CHANGED_ON_DT,
  INLINE_VIEW.AUX1_CHANGED_ON_DT,
  INLINE_VIEW.AUX2_CHANGED_ON_DT,
  INLINE_VIEW.AUX3_CHANGED_ON_DT,
  INLINE_VIEW.AUX4_CHANGED_ON_DT,
  INLINE_VIEW.SRC_EFF_FROM_DT,
  INLINE_VIEW.SRC_EFF_TO_DT,
  CASE WHEN INLINE_VIEW.DELETE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS DELETE_FLG,
  INLINE_VIEW.DATASOURCE_NUM_ID,
  INLINE_VIEW.INTEGRATION_ID,
  INLINE_VIEW.SET_ID,
  INLINE_VIEW.TENANT_ID,
  INLINE_VIEW.X_CUSTOM,
  COALESCE(INLINE_VIEW.SRC_EFF_FROM_DT, to_date('1899-01-01', 'yyyy-MM-dd')) AS EFFECTIVE_FROM_DT,
  to_date('3714-01-01', 'yyyy-MM-dd') AS EFFECTIVE_TO_DT,
  'I' AS IND_UPDATE
FROM (
  SELECT
    MAX(SQ_W_GL_SEGMENT_DS.X_CUSTOM) AS X_CUSTOM,
    SQ_W_GL_SEGMENT_DS.SRC_EFF_TO_DT,
    SQ_W_GL_SEGMENT_DS.DATASOURCE_NUM_ID,
    MAX(SQ_W_GL_SEGMENT_DS.CREATED_ON_DT) AS CREATED_ON_DT,
    MAX(SQ_W_GL_SEGMENT_DS.CHANGED_ON_DT) AS CHANGED_ON_DT,
    MAX(SQ_W_GL_SEGMENT_DS.SEGMENT_VAL_CODE) AS SEGMENT_VAL_CODE,
    MAX(SQ_W_GL_SEGMENT_DS.AUX3_CHANGED_ON_DT) AS AUX3_CHANGED_ON_DT,
    MAX(SQ_W_GL_SEGMENT_DS.AUX1_CHANGED_ON_DT) AS AUX1_CHANGED_ON_DT,
    MAX(SQ_W_GL_SEGMENT_DS.AUX4_CHANGED_ON_DT) AS AUX4_CHANGED_ON_DT,
    MAX(SQ_W_GL_SEGMENT_DS.TENANT_ID) AS TENANT_ID,
    SQ_W_GL_SEGMENT_DS.SRC_EFF_FROM_DT,
    SQ_W_GL_SEGMENT_DS.INTEGRATION_ID,
    MAX(SQ_W_GL_SEGMENT_DS.CREATED_BY_ID) AS CREATED_BY_ID,
    MAX(SQ_W_GL_SEGMENT_DS.DELETE_FLG) AS DELETE_FLG,
    MAX(SQ_W_GL_SEGMENT_DS.SET_ID) AS SET_ID,
    MAX(SQ_W_GL_SEGMENT_DS.SEGMENT_LOV_ID) AS SEGMENT_LOV_ID,
    MAX(SQ_W_GL_SEGMENT_DS.CHANGED_BY_ID) AS CHANGED_BY_ID,
    MAX(SQ_W_GL_SEGMENT_DS.AUX2_CHANGED_ON_DT) AS AUX2_CHANGED_ON_DT,
    NULL AS ROW_WID
  FROM workspace.oracle_edw.W_GL_SEGMENT_DS SQ_W_GL_SEGMENT_DS
  GROUP BY SQ_W_GL_SEGMENT_DS.SRC_EFF_TO_DT, SQ_W_GL_SEGMENT_DS.DATASOURCE_NUM_ID,
    SQ_W_GL_SEGMENT_DS.SRC_EFF_FROM_DT, SQ_W_GL_SEGMENT_DS.INTEGRATION_ID
) INLINE_VIEW
LEFT JOIN workspace.oracle_edw.W_USER_D LKP_USER_CREATED
  ON INLINE_VIEW.DATASOURCE_NUM_ID = LKP_USER_CREATED.DATASOURCE_NUM_ID
  AND INLINE_VIEW.CREATED_BY_ID = LKP_USER_CREATED.INTEGRATION_ID
  AND INLINE_VIEW.CHANGED_ON_DT >= LKP_USER_CREATED.EFFECTIVE_FROM_DT
  AND INLINE_VIEW.CHANGED_ON_DT < LKP_USER_CREATED.EFFECTIVE_TO_DT
  AND LKP_USER_CREATED.DELETE_FLG = 'N'
LEFT JOIN workspace.oracle_edw.W_USER_D LKP_USER_CHANGED
  ON INLINE_VIEW.DATASOURCE_NUM_ID = LKP_USER_CHANGED.DATASOURCE_NUM_ID
  AND INLINE_VIEW.CHANGED_BY_ID = LKP_USER_CHANGED.INTEGRATION_ID
  AND INLINE_VIEW.CHANGED_ON_DT >= LKP_USER_CHANGED.EFFECTIVE_FROM_DT
  AND INLINE_VIEW.CHANGED_ON_DT < LKP_USER_CHANGED.EFFECTIVE_TO_DT
  AND LKP_USER_CHANGED.DELETE_FLG = 'N'
WHERE NOT EXISTS (
  SELECT 1 FROM workspace.oracle_edw.W_GL_SEGMENT_D T
  WHERE T.SRC_EFF_FROM_DT = INLINE_VIEW.SRC_EFF_FROM_DT
    AND T.DATASOURCE_NUM_ID = INLINE_VIEW.DATASOURCE_NUM_ID
    AND T.INTEGRATION_ID = INLINE_VIEW.INTEGRATION_ID
    AND (CASE WHEN T.CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.CHANGED_ON_DT END =
         CASE WHEN INLINE_VIEW.CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.CHANGED_ON_DT END)
    AND (CASE WHEN T.AUX1_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.AUX1_CHANGED_ON_DT END =
         CASE WHEN INLINE_VIEW.AUX1_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.AUX1_CHANGED_ON_DT END)
    AND (CASE WHEN T.AUX2_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.AUX2_CHANGED_ON_DT END =
         CASE WHEN INLINE_VIEW.AUX2_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.AUX2_CHANGED_ON_DT END)
    AND (CASE WHEN T.AUX3_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.AUX3_CHANGED_ON_DT END =
         CASE WHEN INLINE_VIEW.AUX3_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.AUX3_CHANGED_ON_DT END)
    AND (CASE WHEN T.AUX4_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.AUX4_CHANGED_ON_DT END =
         CASE WHEN INLINE_VIEW.AUX4_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.AUX4_CHANGED_ON_DT END)
)

In [ ]:
%sql
-- Step 3: Insert new type 1 values into SCD1 table
INSERT INTO workspace.oracle_edw.W_GL_SEGMENT_T1_D (SCD1_WID, DATASOURCE_NUM_ID, INTEGRATION_ID, W_INSERT_DT, W_UPDATE_DT, ETL_PROC_WID)
SELECT
  row_number() OVER (ORDER BY I.DATASOURCE_NUM_ID, I.INTEGRATION_ID) + COALESCE((SELECT MAX(SCD1_WID) FROM workspace.oracle_edw.W_GL_SEGMENT_T1_D), 0),
  I.DATASOURCE_NUM_ID,
  I.INTEGRATION_ID,
  current_timestamp(),
  current_timestamp(),
  2625027
FROM (
  SELECT DATASOURCE_NUM_ID, INTEGRATION_ID
  FROM I$_2625027_1
  WHERE IND_UPDATE <> 'X'
  GROUP BY DATASOURCE_NUM_ID, INTEGRATION_ID
) I
LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_T1_D T1
  ON T1.DATASOURCE_NUM_ID = I.DATASOURCE_NUM_ID
  AND T1.INTEGRATION_ID = I.INTEGRATION_ID
WHERE T1.SCD1_WID IS NULL

In [ ]:
%sql
-- Step 4: Insert new dimension records
INSERT INTO workspace.oracle_edw.W_GL_SEGMENT_D
(
  ROW_WID,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  SEGMENT_LOV_ID,
  SEGMENT_VAL_CODE,
  SET_ID,
  TENANT_ID,
  X_CUSTOM,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  DELETE_FLG,
  W_INSERT_DT,
  W_UPDATE_DT,
  ETL_PROC_WID,
  SCD1_WID,
  CURRENT_FLG,
  EFFECTIVE_FROM_DT,
  EFFECTIVE_TO_DT
)
SELECT
  row_number() OVER (ORDER BY S.INTEGRATION_ID) + COALESCE((SELECT MAX(ROW_WID) FROM workspace.oracle_edw.W_GL_SEGMENT_D), 0) AS ROW_WID,
  S.DATASOURCE_NUM_ID,
  S.INTEGRATION_ID,
  S.SEGMENT_LOV_ID,
  S.SEGMENT_VAL_CODE,
  S.SET_ID,
  S.TENANT_ID,
  S.X_CUSTOM,
  S.CREATED_BY_WID,
  S.CHANGED_BY_WID,
  S.CREATED_ON_DT,
  S.CHANGED_ON_DT,
  S.AUX1_CHANGED_ON_DT,
  S.AUX2_CHANGED_ON_DT,
  S.AUX3_CHANGED_ON_DT,
  S.AUX4_CHANGED_ON_DT,
  S.SRC_EFF_FROM_DT,
  S.SRC_EFF_TO_DT,
  S.DELETE_FLG,
  current_timestamp(),
  current_timestamp(),
  2625027,
  SCD1.SCD1_WID,
  'Y',
  S.EFFECTIVE_FROM_DT,
  S.EFFECTIVE_TO_DT
FROM I$_2625027_1 S
INNER JOIN workspace.oracle_edw.W_GL_SEGMENT_T1_D SCD1
  ON SCD1.DATASOURCE_NUM_ID = S.DATASOURCE_NUM_ID
  AND SCD1.INTEGRATION_ID = S.INTEGRATION_ID
WHERE S.IND_UPDATE IN ('I', 'S')

In [ ]:
%sql
-- Step 5: Fix effective dates for historized records
MERGE INTO workspace.oracle_edw.W_GL_SEGMENT_D AS T
USING (
  SELECT
    T0.ROW_WID,
    COALESCE(MIN(T1.EFFECTIVE_FROM_DT), to_date('3714-01-01', 'yyyy-MM-dd')) AS NEW_EFFECTIVE_TO_DT,
    CASE WHEN MIN(T1.EFFECTIVE_FROM_DT) IS NULL THEN 'Y' ELSE 'N' END AS NEW_CURRENT_FLG
  FROM workspace.oracle_edw.W_GL_SEGMENT_D T0
  INNER JOIN workspace.oracle_edw.W_GL_SEGMENT_D T1
    ON T0.DATASOURCE_NUM_ID = T1.DATASOURCE_NUM_ID
    AND T0.INTEGRATION_ID = T1.INTEGRATION_ID
    AND T0.EFFECTIVE_FROM_DT < T1.EFFECTIVE_FROM_DT
    AND T1.DELETE_FLG = 'N'
  WHERE T1.ETL_PROC_WID = 2625027
    AND T0.DELETE_FLG = 'N'
  GROUP BY T0.ROW_WID
) SRC
ON T.ROW_WID = SRC.ROW_WID
WHEN MATCHED THEN UPDATE SET
  T.EFFECTIVE_TO_DT = SRC.NEW_EFFECTIVE_TO_DT,
  T.CURRENT_FLG = SRC.NEW_CURRENT_FLG

In [ ]:
%sql
-- Step 6: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT 380 AS DATASOURCE_NUM_ID, 'SILOS_SIL_GLSEGMENTDIMENSION' AS PACKAGE_NAME,
    'W_GL_SEGMENT_D' AS TARGET_TABLE_NAME, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2625027 AS ETL_PROC_WID, 12345 AS LOAD_PLAN_ID, date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    (SELECT CASE WHEN IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM VAR_IS_INCREMENTAL) AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN UPDATE SET TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME, TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
  TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID, TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
  TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE, TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN INSERT (DATASOURCE_NUM_ID, PACKAGE_NAME, TARGET_TABLE_NAME, ETL_USAGE_CODE,
  ETL_PROC_WID, LOAD_PLAN_ID, WIP_LOAD_START_DATE, ETL_LOAD_DATE, COMMITTED)
VALUES (SRC.DATASOURCE_NUM_ID, SRC.PACKAGE_NAME, SRC.TARGET_TABLE_NAME, SRC.ETL_USAGE_CODE,
  SRC.ETL_PROC_WID, SRC.LOAD_PLAN_ID, SRC.WIP_LOAD_START_DATE, SRC.ETL_LOAD_DATE, SRC.COMMITTED)

In [ ]:
%sql
-- Step 7: Insert reference dates to History Table
INSERT INTO workspace.oracle_edw.W_ETL_LOAD_DATES_LOG
SELECT DATASOURCE_NUM_ID, PACKAGE_NAME, TARGET_TABLE_NAME, ETL_USAGE_CODE, ETL_PROC_WID, LOAD_PLAN_ID,
  2625027 AS SESSION_ID, WIP_LOAD_START_DATE, LAST_MAX_DATE, ETL_LOAD_DATE, COMMITTED
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380 AND PACKAGE_NAME = 'SILOS_SIL_GLSEGMENTDIMENSION' AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 8: Drop flow table
DROP VIEW IF EXISTS I$_2625027_1;
DROP VIEW IF EXISTS VAR_IS_INCREMENTAL